## 1. CandleRule中的相关配置

与"维护实时历史K线功能"重要相关配置为：


|参数|类型|默认值|说明|
|:---|:---|:---|:---|
|**SERVER_WEIGHT**|float, int|0.75|服务占用权重上限的比例 现货交易权重上限1200 合约交易权重上限2400|
|**SYMBOLS**|str,list|'all'|收集的产品 all表示全部正在交易的产品|
|**SYMBOLS_FILTER**|list|[]|过滤的产品|
|**SYMBOL_CONTAINS**|str|''|产品名称中需要包含的内容|
|**SYMBOL_ENDSWITH**|str|''|产品名称必须以何内容结尾|
|**BAR**|str|binance_interface.binace_lite.BINANCE_DEFAULT_BAR|时间颗粒度|
|**TIMEZONE**|str|binance_interface.binace_lite.BINANCE_TIMEZONE|时区|
|**CANDLE_DIR**|str|binance_interface.binace_lite.BINANCE_CANDLE_DATE_BASE_DIR|以天为单位的数据存储路径|
|**DOWNLOAD_TIME**|str, None|'00:10:00'|每日下载昨日历史K线数据的时刻 格式：%H:%M:%S None表示不下载（每日定时下载K线中使用）|
|**LENGTH**|int|1440 * 2|candle_map 缓存K线数据长度（实时K线服务）|
|**UPDATE_INTERVAL_SECONDS**|int|3|candle_map 更新时间间隔（秒）（实时K线服务）|
|**CACHE_DELAY_SECONDS**|int, None|60 * 60|candle_map 保存缓存文件时间间隔（秒） None：不保存缓存（实时K线服务）|
|**CACHE_DIR**|str|'./BINANCE_CACHE'|缓存数据路径（实时K线服务）|
|**LOCAL_CANDLE_DAYS**|int|2|服务启动 需维护本地多少天的历史K线数据（实时K线服务）|

## 2. 重新定义规则

In [1]:
from binance_interface.app.candle_server import CandleRule,CandleServer

In [2]:
class MyCandleRule(CandleRule):
    # 收集的产品 all表示全部正在交易的产品
    SYMBOLS = [
        'BTCUSDT',
        'ETHUSDT',
    ]
    # candle_map 缓存K线数据长度
    LENGTH = 1440
    # 时间颗粒度
    BAR = '1m'
    # 每日下载昨日历史K线数据的时刻 格式：%H:%M:%S None表示不下载
    DOWNLOAD_TIME = None  # 如果你维护的实时K线长度不足以包含一整天，应该关闭DOWNLOAD_TIME功能

## 3. 启动实时K线维护

In [3]:
# 币币交易：SPOT；U本位合约：UM；币本位合约：CM
instType = 'SPOT'
# 永续合约，默认规则
candleServer = CandleServer(
    instType=instType,
    rule=MyCandleRule,
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/binance_resender
    proxy_host=None,
)
# 启动K线维护服务（堵塞直到candle_map维护完成）
candleServer.run_candle_map()
# 被异步维护的candle_map字典
candleServer.candle_map

2024-01-26 01:24:21,864 [INFO] DOWNLOAD BTCUSDT    1m  24/01/2024
2024-01-26 01:24:22,461 [INFO] DOWNLOAD ETHUSDT    1m  24/01/2024
2024-01-26 01:24:22,463 [INFO] COMPLETE DOWNLOAD 24/01/2024 (ALL:2 SKIP:0 SUC:2 WARN:0 ERROR:0)
2024-01-26 01:24:22,606 [INFO] COMPLETE DOWNLOAD 23/01/2024 (ALL:2 SKIP:2 SUC:0 WARN:0 ERROR:0)
2024-01-26 01:24:24,614 [INFO] COMPLETE PREPARE candle_map
2024-01-26 01:24:25,119 [INFO] DOWNLOAD CANDLE CACHE


{'BTCUSDT': array([[1.70611710e+12, 4.01380300e+04, 4.01498600e+04, ...,
         3.90161400e+01, 1.56539079e+06, 0.00000000e+00],
        [1.70611716e+12, 4.01231600e+04, 4.01270400e+04, ...,
         2.72394000e+00, 1.09286470e+05, 0.00000000e+00],
        [1.70611722e+12, 4.00949600e+04, 4.00949600e+04, ...,
         1.44322000e+00, 5.78375603e+04, 0.00000000e+00],
        ...,
        [1.70620332e+12, 3.96406700e+04, 3.96580000e+04, ...,
         1.22753300e+01, 4.86592058e+05, 0.00000000e+00],
        [1.70620338e+12, 3.96580000e+04, 3.96660000e+04, ...,
         1.30728800e+01, 5.18430454e+05, 0.00000000e+00],
        [1.70620344e+12, 3.96659300e+04, 3.96659400e+04, ...,
         1.49545000e+00, 5.93081059e+04, 0.00000000e+00]]),
 'ETHUSDT': array([[1.70611710e+12, 2.23935000e+03, 2.24040000e+03, ...,
         1.05016800e+02, 2.35143903e+05, 0.00000000e+00],
        [1.70611716e+12, 2.23840000e+03, 2.23841000e+03, ...,
         2.52323000e+01, 5.64626045e+04, 0.00000000e+00],
   

## 4. 模拟实盘交易实时获取K线

In [4]:
import time
import datetime

# 打印实时更新的candle
for i in range(6):
    # 以安全的方式从candle_map中获取历史线，K线最新时间不能小于当前时间戳大于security_seconds秒
    candle = candleServer.get_candle_security('BTCUSDT', security_seconds=60 * 3)
    print(
        'K线长度:',
        candle.shape[0],
        'K线的最新时间:',
        datetime.datetime.fromtimestamp(int(candle[-1, 0] / 1000)),
        '当前时间:',
        datetime.datetime.now()
    )
    print(candle)  # 类型：np.ndarray
    time.sleep(10)

K线长度: 1440 K线的最新时间: 2024-01-26 01:24:00 当前时间: 2024-01-26 01:24:25.634994
[[1.70611710e+12 4.01380300e+04 4.01498600e+04 ... 3.90161400e+01
  1.56539079e+06 0.00000000e+00]
 [1.70611716e+12 4.01231600e+04 4.01270400e+04 ... 2.72394000e+00
  1.09286470e+05 0.00000000e+00]
 [1.70611722e+12 4.00949600e+04 4.00949600e+04 ... 1.44322000e+00
  5.78375603e+04 0.00000000e+00]
 ...
 [1.70620332e+12 3.96406700e+04 3.96580000e+04 ... 1.22753300e+01
  4.86592058e+05 0.00000000e+00]
 [1.70620338e+12 3.96580000e+04 3.96660000e+04 ... 1.30728800e+01
  5.18430454e+05 0.00000000e+00]
 [1.70620344e+12 3.96659300e+04 3.96659400e+04 ... 1.49545000e+00
  5.93081059e+04 0.00000000e+00]]
K线长度: 1440 K线的最新时间: 2024-01-26 01:24:00 当前时间: 2024-01-26 01:24:35.635139
[[1.70611710e+12 4.01380300e+04 4.01498600e+04 ... 3.90161400e+01
  1.56539079e+06 0.00000000e+00]
 [1.70611716e+12 4.01231600e+04 4.01270400e+04 ... 2.72394000e+00
  1.09286470e+05 0.00000000e+00]
 [1.70611722e+12 4.00949600e+04 4.00949600e+04 ... 1.443

## 5. 关闭服务

In [5]:
# 关闭服务
candleServer.close_run_candle_map()